In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import  array_to_img, load_img, img_to_array
from keras import callbacks
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.optimizers import adam
from keras.models import load_model
import cv2
import os
import gc



from numpy.random import seed
seed(2019)
#from tensorflow import set_random_seed
#set_random_seed(2019)

Using TensorFlow backend.


In [2]:
path = r"../input/images/task_images"

train_files = pd.read_csv(r'../input/convertcsv.csv') 


#convertcsv = pd.read_csv("../input/convertcsv.csv")

In [3]:
train_files.replace(['product'],1,inplace = True)
train_files.replace(['person with product'],0,inplace = True)

In [4]:

train_files['Label'].value_counts()

1    379
0    212
Name: Label, dtype: int64

In [5]:
#train_files.replace(['product'],1,inplace = True)

In [6]:
def load_images(train):
    arr = []
    for file in tqdm(train_files['Image'].values):
        img = load_img(os.path.join(path, file), target_size=(224,224))
        img = img_to_array(img)/255. # normalize image 
        arr.append(img)

    gc.collect()
    return np.asarray(arr)

In [7]:
X_train = load_images(train_files)

100%|██████████| 591/591 [00:10<00:00, 54.04it/s]


In [8]:
X_train

array([[[[0.13333334, 0.13333334, 0.24313726],
         [0.13333334, 0.13333334, 0.24313726],
         [0.13333334, 0.13333334, 0.24313726],
         ...,
         [0.73333335, 0.59607846, 0.4862745 ],
         [0.72156864, 0.5803922 , 0.47843137],
         [0.70980394, 0.56078434, 0.47058824]],

        [[0.11372549, 0.12941177, 0.23529412],
         [0.11372549, 0.12941177, 0.23529412],
         [0.10980392, 0.1254902 , 0.23137255],
         ...,
         [0.69803923, 0.57254905, 0.48235294],
         [0.6745098 , 0.5529412 , 0.47058824],
         [0.6509804 , 0.5294118 , 0.4509804 ]],

        [[0.10980392, 0.13333334, 0.23529412],
         [0.10980392, 0.13333334, 0.23529412],
         [0.10588235, 0.12941177, 0.23137255],
         ...,
         [0.6431373 , 0.5294118 , 0.45882353],
         [0.62352943, 0.5176471 , 0.4509804 ],
         [0.5921569 , 0.4862745 , 0.41960785]],

        ...,

        [[0.18431373, 0.15294118, 0.23529412],
         [0.22352941, 0.1764706 , 0.25490198]

In [9]:
from sklearn.preprocessing import OneHotEncoder
one_hot = OneHotEncoder(dtype='int8', sparse=False)
y_train = train_files['Label']

In [10]:
X_train.shape

(591, 224, 224, 3)

In [11]:
from keras.applications.densenet import DenseNet121
model = DenseNet121(include_top=False, input_shape=(224,224,3))

29089792/29084464 [==============================] - 1s 0us/step


In [12]:

from keras.models import Model
from keras.layers import Dense,Activation, GlobalAveragePooling2D

from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

x = GlobalAveragePooling2D()(model.output)
output = Dense(1, activation='sigmoid')(x)
model.summary()
model = Model(model.inputs, output)
model.save('model.hdf5')

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [13]:
#from keras.applications.resnet50 import ResNet50
#model_1 = ResNet50(include_top=False, input_shape=(224,224,3))

In [14]:
from keras.layers import Dense,Activation

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, 
                                                    stratify=y_train,
                                                    random_state=42,
                                                    test_size=0.2)

In [17]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 16
epochs = 10


model.compile(loss='binary_crossentropy', metrics=['accuracy'],
              optimizer=adam(lr=0.0001))

datagen = ImageDataGenerator(rotation_range=45, 
                             horizontal_flip=True, 
                             width_shift_range=0.5, 
                             height_shift_range=0.5, 
                             dtype='float32')

datagen.fit(X_train, augment=True, rounds=1, seed=2019)
train_generator = datagen.flow(X_train, y_train, 
                               batch_size=batch_size, 
                               seed=2019)


cp = ModelCheckpoint('best.hdf5', monitor='val_loss', save_best_only=True)
annealer = LearningRateScheduler(lambda x: 1e-4 * 0.95 ** x)

history = model.fit_generator(generator=train_generator, 
                              steps_per_epoch=len(X_train)/batch_size, 
                              validation_data=[X_test, y_test], 
                              callbacks=[cp],
                              epochs=8)

Epoch 1/8
30/29 [==============================] - 85s 3s/step - loss: 0.5199 - accuracy: 0.7352 - val_loss: 0.3731 - val_accuracy: 0.8151
Epoch 2/8
30/29 [==============================] - 10s 348ms/step - loss: 0.3942 - accuracy: 0.8220 - val_loss: 0.3435 - val_accuracy: 0.8403
Epoch 3/8
30/29 [==============================] - 10s 339ms/step - loss: 0.3145 - accuracy: 0.8708 - val_loss: 0.4088 - val_accuracy: 0.8319
Epoch 4/8
30/29 [==============================] - 11s 351ms/step - loss: 0.3083 - accuracy: 0.8771 - val_loss: 0.2785 - val_accuracy: 0.8739
Epoch 5/8
30/29 [==============================] - 10s 344ms/step - loss: 0.2578 - accuracy: 0.9047 - val_loss: 0.2905 - val_accuracy: 0.8824
Epoch 6/8
30/29 [==============================] - 10s 347ms/step - loss: 0.2111 - accuracy: 0.9195 - val_loss: 0.2866 - val_accuracy: 0.8403
Epoch 7/8
30/29 [==============================] - 10s 347ms/step - loss: 0.1773 - accuracy: 0.9280 - val_loss: 0.2745 - val_accuracy: 0.9328
Epoch 8/8

In [18]:
y_pred = model.predict(X_test)

In [19]:
y_pred_train = model.predict(X_train)

In [20]:
#y_pred

In [21]:
from termcolor import colored
#from sklearn.cross_validation import KFold

from sklearn.metrics import confusion_matrix

In [22]:

# Making the Confusion Matrix
cm = confusion_matrix(y_test, (y_pred>0.5))
print(colored('The Confusion Matrix is: ', 'red'),'\n', cm)
# Calculate the accuracy on test set
predict_accuracy_on_test_set = (cm[0,0] + cm[1,1])/(cm[0,0] + cm[1,1]+cm[1,0] + cm[0,1])
print(colored('The Accuracy on Test Set is: ', 'blue'), colored(predict_accuracy_on_test_set, 'blue'))

The Confusion Matrix is:  
 [[29 14]
 [ 0 76]]
The Accuracy on Test Set is:  0.8823529411764706


In [23]:
np.savetxt("label_Dense.csv", y_pred, delimiter=",")

In [24]:
np.savetxt("train_Dense.csv", y_pred, delimiter=",")

In [25]:
np.savetxt("train_label_Dense.csv", y_train, delimiter=",")